# Package import

In [3]:
import pandas as pd
import os 
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from collections import defaultdict
from sklearn.decomposition import PCA
import PyPDF2
from sklearn.metrics import silhouette_score
import cv2
import numpy as np
import matplotlib.pyplot as plt
try:
    from PIL import Image
except ImportError:
    import Image
import pytesseract

# If you don't have tesseract executable in your PATH, include the following:
pytesseract.pytesseract.tesseract_cmd = r'C:\Program Files\Tesseract-OCR\tesseract.exe'


# Create List

In [4]:
df = pd.read_excel(r'source_docs')

In [6]:
df2 = pd.read_excel(r'file_path')

In [7]:
df_merge = pd.merge(df,df2, on=['ReferralName','ReferralCode','Source'],how='inner')

In [8]:
df_merge['FilePath']=df_merge['FilePath'].apply(lambda x: x.replace("Faxes","Z:"))

In [10]:
df_path = df_merge['FilePath']
pdf_path = df_path.values.tolist()

In [11]:
print(len(pdf_path))

8786


# Transform PDF to PNG

In [17]:
from wand.image import Image
import pytesseract

# Set the desired resolution (DPI)
dpi = 200

destination = r'C:\Users\sanghoon.pai\OneDrive - AdaptHealth\Desktop\sang-adapt-dojo\bucket6_png'
# Open the PDF file and change it to png
for pt in pdf_path[1205:]:
    with Image(filename=pt, resolution=dpi) as img:
        file_name = destination + "\\" +os.path.basename(pt).split(".")[0] + ".png"
        try:
            img.save(filename=file_name)
        except:
            pass

In [16]:
# code to get index where code being stopped and need to resume from the point
pdf_name=''
index = [i for i, path in enumerate(pdf_path) if pdf_name in path]
print(index)

[1205]


# list png

In [ ]:
# png_path = []
# png_name = []

# def list_files(path):
#     for filename in os.listdir(path):
#         file_path = os.path.join(path, filename)
#         if os.path.isfile(file_path):
#             png_path.append(file_path)
#             png_name.append(filename)

# list_files(r"C:\Users\sanghoon.pai\OneDrive - AdaptHealth\Desktop\sang-adapt-dojo\bucket6_png")
# png_dict = dict(zip(png_name, png_path))

# Image Preprocessing

In [ ]:
def image_preprocessing(png, destination):
    image = cv2.imread(png, cv2.IMREAD_GRAYSCALE)
    _, binary_image = cv2.threshold(image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    # Deskew the image using the skew angle
    coords = np.column_stack(np.where(binary_image > 0))
    angle = cv2.minAreaRect(coords)[-1]
    if angle > 45:
        angle = 90 - angle
    else:
        angle = -angle

    (height, width) = binary_image.shape
    center = (width // 2, height // 2)
    M = cv2.getRotationMatrix2D(center, angle, 1.0)
    deskewed_image = cv2.warpAffine(binary_image, M, (width, height), flags=cv2.INTER_CUBIC, borderMode=cv2.BORDER_REPLICATE)

    resized_image = cv2.resize(deskewed_image, (1920, 1920))
    kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    sharpened_image = cv2.filter2D(resized_image, -1, kernel)

    file_name = destination + "\\" +os.path.basename(pt).split(".")[0] + ".png"
    # Save the pre-processed image
    cv2.imwrite(file_name, sharpened_image)

In [ ]:
# destination = r'C:\Users\sanghoon.pai\OneDrive - AdaptHealth\Desktop\sang-adapt-dojo\bucket6_png_processed'
# for png in png_path:
#     image_preprocessing(png,destination)

# Text Extraction

In [ ]:
png_path = []
png_name = []

def list_files(path):
    for filename in os.listdir(path):
        file_path = os.path.join(path, filename)
        if os.path.isfile(file_path):
            png_path.append(file_path)
            png_name.append(filename)
list_files(r"C:\Users\sanghoon.pai\OneDrive - AdaptHealth\Desktop\sang-adapt-dojo\bucket6_png")
png_dict = dict(zip(png_name, png_path))

from PIL import Image
import pytesseract
full_text = {}
for i, pt in enumerate(png_dict):
    try:
        image = Image.open(r"C:\Users\sanghoon.pai\OneDrive - AdaptHealth\Desktop\sang-adapt-dojo\bucket6_png" + pt)
        text = pytesseract.image_to_string(image)
        full_text[pt] = text
    except:
        pass

: 

In [ ]:
import json
with open('full_text_bucket6.json','w') as f:
    json.dump(full_text,f)

In [ ]:
df3= pd.DataFrame.from_dict(png_dict , orient='index')
df3=df3.reset_index()
df3=df3.rename(columns={df3.columns[0]:'doc_name',df3.columns[1]: 'text'})
df3['group'] = df3['doc_name'].str.split('-').str[0]
result = df3.groupby('group').agg({'text': ' '.join})
result= result.reset_index()
result.columns = ['doc_name','text']
result['doc_name'] = result['doc_name'].apply(lambda x: x + '.png' if not x.endswith('.png') else x)
result.to_csv('bucket6.csv')

# SK-Learn Kmeans clustering

In [ ]:
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(full_text)


# Use elbow method to find the optimal number of clusters
inertias = []
silhouette_scores=[]
for k in range(2, 2000,50):
    kmeans = KMeans(n_clusters=k)
    kmeans.fit(tfidf_matrix)
    inertias.append(kmeans.inertia_)
    silhouette_scores.append(silhouette_score(tfidf_matrix, kmeans.labels_))

In [ ]:
# Based on the elbow plot, choose the optimal number of clusters
n_clusters = 700
kmeans = KMeans(n_clusters=n_clusters)
clusters = kmeans.fit_predict(tfidf_matrix)

# Use silhouette method to evaluate the clustering performance
silhouette_avg = silhouette_score(tfidf_matrix, clusters)
print("For n_clusters =", n_clusters, "The average silhouette_score is :", silhouette_avg)

# Create dataframe with clustering results


cluster_pd = pd.DataFrame(clusters, columns = ['cluster'])
name=pd.DataFrame(doc_name, columns = ['doc_name'])
cluster_pd['doc_name']=doc_name

cluster_pd.sort_values(by=["cluster"], inplace = True)


In [ ]:

cluster_pd = pd.DataFrame(clusters, columns = ['cluster'])
cluster_pd.sort_values(by=["cluster"], inplace = True)
cluster_merge = cluster_pd.merge(result,left_index=True, right_index=True,how='outer')
cluster_merge.sort_values(by=["cluster"], inplace = True)
cluster_merge['doc_name'] = cluster_merge['doc_name'].str.replace( '.png','.pdf')
df3.to_csv('train_cluster700.csv')

In [ ]:
import joblib
joblib.dump(kmeans, "kmeans_train_ocr.joblib")

import pickle
with open("kmeans.pkl", "wb") as file:
    pickle.dump(kmeans, file)